# FIFA World Cup 2026 Style Smoke Test (R)

This notebook models the finals tournament structure:

- 48 teams
- 12 groups of 4 (single round-robin)
- Advance 32 teams: top 2 from each group + best 8 third-placed teams
- Knockout: Round of 32 -> Round of 16 -> Quarter-finals -> Semi-finals -> Third-place -> Final
- Total matches: 72 (groups) + 32 (knockout with third-place) = 104

Notes:
- Uses `bracketeer` grouped standings (`group`, `group_rank`, `points`, `score_diff`) to rank third-placed teams.
- This smoke test targets tournament architecture and end-to-end progression.


In [ ]:
if (!requireNamespace("bracketeer", quietly = TRUE)) {
  stop("Please install bracketeer first: devtools::install('.')")
}

library(bracketeer)


In [ ]:
teams <- c(
  "Canada", "Mexico", "United States",
  "Argentina", "Brazil", "Uruguay", "Colombia", "Ecuador", "Paraguay", "Chile",
  "England", "France", "Germany", "Spain", "Portugal", "Netherlands", "Belgium", "Croatia",
  "Italy", "Denmark", "Switzerland", "Austria", "Serbia", "Poland", "Ukraine",
  "Japan", "Korea Republic", "IR Iran", "Saudi Arabia", "Australia", "Qatar", "Uzbekistan", "Jordan",
  "Morocco", "Senegal", "Tunisia", "Algeria", "Egypt", "Cote d'Ivoire", "Ghana", "Cameroon",
  "Nigeria", "South Africa",
  "New Zealand",
  "Costa Rica", "Panama", "Jamaica", "Honduras"
)

stopifnot(length(teams) == 48L)

# Selector for World Cup progression:
# - automatic qualifiers: top 2 per group (24)
# - additional qualifiers: best 8 third-placed teams (8)
best_32_selector <- filter_by(function(standings = NULL, source_pool = NULL, ...) {
  if (is.null(standings) || !is.data.frame(standings) ||
      !all(c("participant", "group") %in% names(standings))) {
    pool <- as.character(source_pool)
    return(head(pool, min(length(pool), 32L)))
  }

  table <- standings
  table$participant <- as.character(table$participant)
  table$group <- as.character(table$group)

  if (!"rank" %in% names(table)) {
    table$rank <- seq_len(nrow(table))
  }
  table$rank <- as.integer(table$rank)

  if (!"group_rank" %in% names(table)) {
    table <- table[order(table$group, table$rank), , drop = FALSE]
    table$group_rank <- ave(
      table$rank,
      table$group,
      FUN = function(x) as.integer(rank(x, ties.method = "first"))
    )
  }

  if (!"points" %in% names(table)) table$points <- 0
  if (!"score_diff" %in% names(table)) table$score_diff <- 0

  top_two <- table[table$group_rank <= 2L, , drop = FALSE]
  thirds <- table[table$group_rank == 3L, , drop = FALSE]
  thirds <- thirds[order(-thirds$points, -thirds$score_diff, thirds$rank), , drop = FALSE]

  selected <- c(
    as.character(top_two$participant),
    as.character(utils::head(thirds$participant, 8L))
  )
  selected <- unique(selected)

  if (length(selected) != 32L) {
    stop("Selector expected 32 qualifiers, got ", length(selected))
  }

  selected
})

wc2026_spec <- spec() |>
  round_robin(
    "groups",
    groups = 12,
    tiebreakers = c("points", "score_diff", "sos", "head_to_head", "alphabetical")
  ) |>
  single_elim("knockout", third_place = TRUE, take = best_32_selector)

preflight <- validate(wc2026_spec, n = length(teams))
stopifnot(isTRUE(preflight$ok))

trn <- build(wc2026_spec, teams)

initial_status <- stage_status(trn)
stopifnot(initial_status$total[initial_status$stage == "groups"] == 72L)
stopifnot(initial_status$total[initial_status$stage == "knockout"] == 0L)

initial_status


In [ ]:
set.seed(2026)

random_score <- function(max_goals = 4L, allow_draw = TRUE) {
  score <- sample.int(max_goals + 1L, size = 2L, replace = TRUE) - 1L
  if (!allow_draw && score[[1]] == score[[2]]) {
    winner_idx <- sample.int(2L, size = 1L)
    score[[winner_idx]] <- score[[winner_idx]] + 1L
  }
  as.numeric(score)
}

# Complete group stage: 72 matches
group_matches <- matches(trn, "groups", status = "all")
stopifnot(nrow(group_matches) == 72L)

for (i in seq_len(nrow(group_matches))) {
  trn <- result(
    trn,
    stage = "groups",
    match = group_matches$match_id[[i]],
    score = random_score(max_goals = 4L, allow_draw = TRUE),
    auto_advance = FALSE
  )
}

# Materialize Round of 32 from top-2 + best-8 third-placed teams
trn <- advance(trn, stage = "groups")

after_groups <- stage_status(trn)
stopifnot(after_groups$materialized[after_groups$stage == "knockout"])
stopifnot(after_groups$total[after_groups$stage == "knockout"] == 32L)
stopifnot(nrow(matches(trn, "knockout", status = "all")) == 32L)

# Confirm 32 unique knockout entrants
k_all <- matches(trn, "knockout", status = "all")
knockout_participants <- sort(unique(na.omit(c(k_all$participant1, k_all$participant2))))
stopifnot(length(knockout_participants) == 32L)

after_groups


In [ ]:
# Play knockout until champion is decided
for (step in seq_len(64L)) {
  if (isTRUE(trn$completed)) break

  pending <- matches(trn, "knockout", status = "pending")
  ready <- pending[!is.na(pending$participant1) & !is.na(pending$participant2), , drop = FALSE]
  if (nrow(ready) == 0L) {
    stop("No playable knockout matches are available")
  }

  for (mid in ready$match_id) {
    trn <- result(
      trn,
      stage = "knockout",
      match = mid,
      score = random_score(max_goals = 4L, allow_draw = FALSE),
      auto_advance = FALSE
    )
  }
}

stopifnot(isTRUE(trn$completed))
champion <- winner(trn)
stopifnot(!is.na(champion))

total_matches <- nrow(matches(trn, status = "all"))
stopifnot(total_matches == 104L)

cat("FIFA 2026-style smoke test passed. Champion:", champion, "\n")
cat("Total matches in tournament:", total_matches, "\n")
